In [2]:
import json
import os
import random
from pathlib import Path

import pandas as pd
from datasets import load_dataset
from dotenv import load_dotenv

load_dotenv()

EXTERNAL = Path(os.getenv("EXTERNAL_STORAGE_DIR"))
DATA_DIR = Path(os.getenv("DATA_DIR", "data"))
PROC_DIR = DATA_DIR / "processed"
SPLIT_DIR = DATA_DIR / "splits"

PROC_DIR.mkdir(parents=True, exist_ok=True)
SPLIT_DIR.mkdir(parents=True, exist_ok=True)

print(f"EXTERNAL_STORAGE_DIR: {EXTERNAL}")
print(f"DATA_DIR: {DATA_DIR.resolve()}")

EXTERNAL_STORAGE_DIR: /Users/daria/Documents/files_drive/llm-news-summarizer-ru
DATA_DIR: /Users/daria/Documents/my_projects/llm-news-summarizer-ru/data


In [3]:
ds = load_dataset("IlyaGusev/gazeta")

ds

Generating test split: 100%|██████████████████████████████████████████████| 6793/6793 [00:00<00:00, 39388.34 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
})

In [4]:
ds["train"][0]

{'text': 'Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, автомобилисты будут понимать, за что они платят, а дорожники будут иметь гарантированный доход. Кроме налога дорожные фонды будут пополняться за счет бюджетных средств и проезда по платным дорогам. Более того, транспортный налог предлагается завуалировать в акцизы на бензин. Привычную и раздражающую систему ежегодной оплаты квитанции предлагается изменить, включив налог в стоимость топлива. Минэкономразвития говорит об удвоении акцизы, которая сегодня состав

In [13]:
splits = list(ds.keys())
features = list(ds[splits[1]].features.keys())
print(f"Feature names: {features}")
print(f"Splits: {splits}")

Feature names: ['text', 'summary', 'title', 'date', 'url']
Splits: ['train', 'validation', 'test']


In [12]:
text_col, summ_col, title_col = features[:3]

In [19]:
from typing import Optional


def clean(s: Optional[pd.Series]) -> Optional[pd.Series]:
    s = s.fillna("")
    s = (
        s.str.replace("\xa0", " ", regex=False)
        .str.replace("\u2009", " ", regex=False)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )
    return s

In [23]:
for split in splits:
    df = ds[split].to_pandas()
    df["origin_idx"] = df.index
    df[title_col] = clean(df[title_col])
    df[text_col] = clean(df[text_col])
    df[summ_col] = clean(df[summ_col])
    df = df[df[text_col].str.len() > 300].reset_index(drop=True)
    out = pd.DataFrame(
        {
            "id": (split + "_" + df["origin_idx"].astype(str)),
            "title": df[title_col],
            "text": df[text_col],
            "reference_summary": df[summ_col],
        }
    )

    # out_split_ids = SPLIT_DIR / f"gazeta_{split}_ids.csv"
    # out["id"].to_csv(out_split_ids, index=False, header=False)

    # out_split_sample = SPLIT_DIR / f"gazeta_{split}_sample.jsonl"
    # sample = out.sample(n=min(500, len(out)), random_state=42)
    # sample.to_json(out_split_sample, orient="records", force_ascii=False, lines=True)

    out_proc = PROC_DIR / f"gazeta_{split}.jsonl"
    out.to_json(out_proc, orient="records", force_ascii=False, lines=True)

    # if EXTERNAL:
    #     mirror = EXTERNAL / "data" / "raw"
    #     mirror.mkdir(parents=True, exist_ok=True)
    #     out.to_json(
    #         mirror / f"gazeta_{split}.jsonl",
    #         orient="records",
    #         force_ascii=False,
    #         lines=True,
    #     )